In [1]:
import os

print('Process (%s) start...' % os.getpid())

# pid = os.fork()
# if pid == 0:
#     print('I am child process (%s) and my parent is (%s)' % (os.getpid(), os.getpid()))
# else:
#     print('I just create a child process (%s)' % (os.getpid()))


Process (4454) start...


In [3]:
import multiprocessing as mp
import os

#   子进程需要执行的代码
def run_proc(name):
    print('Run child process (%s) (%s)...' % (name, os.getpid()))

if __name__ == '__main__':
    print('Parent process (%s)...' % os.getpid())
    p = mp.Process(target=run_proc, args=('test',))
    print('Child process will start...')
    p.start()
    p.join()
    print('Child process end.')

Parent process (4454)...
Child process will start...


Run child process (test) (4794)...


Child process end.


In [6]:
#   如果需要启动大量的子进程，　可以使用进程池的方式批量创建子进程

import multiprocessing as mp
import os, time, random

def long_time_task(name):
    print('Run task %s (%s)...' % (name, os.getpid()))
    start = time.time()
    time.sleep(random.random() * 3)
    end = time.time()
    print('Tast %s runs %0.2f seconds.' % (name, (end-start)))
    
if __name__ == '__main__':
    print('Parent process (%s).' % (os.getpid()))
    p = mp.Pool(8)  # CPU 核数
    for i in range(9):  # 最多支持最大CPU核数进程并行运算
        p.apply_async(long_time_task, args=(i,))
    print('Waiting for all process done')
    p.close()
    p.join()
    print('all process done')


Parent process (4454).


Run task 1 (5230)...


Run task 3 (5232)...


Run task 2 (5231)...


Run task 6 (5235)...


...

Tast 0 runs 1.58 seconds.


Tast 5 runs 2.00 seconds.


Tast 7 runs 2.18 seconds.


Tast 8 runs 2.71 seconds.


all process done


In [11]:
#   Process 之间的通信
#   通信方式有Queue, Pipes等
#   以Queue为例,　一个进程往Queue写数据, 一个从Queue里读数据

import multiprocessing as mp
import os, time, random

def write(q):
    print('Process to write: %s' % os.getpid())
    for value in ['A', 'B', 'C']:
        print('Put %s to queue...' % value)
        q.put(value)
        time.sleep(random.random())

def read(q):
    print('Process to read: %s' % os.getpid())
    while True:
        value = q.get(True)
        print('Get %s from queue.' % value)
        
if __name__ == '__main__':
    q = mp.Queue()
    pw = mp.Process(target=write, args=(q,))
    pr = mp.Process(target=read, args=(q,))
    pw.start()
    pr.start()
    pw.join()
    #   pr进程里是死循环，　无法等待其结束，只能强行终止
    pr.terminate()

Process to write: 6049


Put A to queue...


Put B to queue...


Process to read: 6050


Put C to queue...


Get A from queue.


In [21]:
import multiprocessing as mp
import os

def write1(q):
    res = 100
    q.put(res)
    print('w1 finished')
    
def write2(q):
    res = 200
    q.put(res)
    print('w2 finished')
    
if __name__ == '__main__':
    q = mp.Queue()
    w1 = mp.Process(target=write1, args=(q,))
    w2 = mp.Process(target=write2, args=(q,))
    
    w1.start()
    w2.start()
    w1.join()
    w2.join()
    res1 = q.get()  #   Queue: first in first out
    # res2 = q.get()
    print(res1)
    

w1 finished


w2 finished


100


In [64]:
import multiprocessing as mp
import threading as td
import time

"""
语法解释：

try:

     表达式 1（如果表达式，可以成功执行，则执行，跳到 finally 语句）

except ExpectErrorType, Argument:   （表达式1 没有做成功，且正好是 ExpectErrorType 的错误情况，则执行）

     表达式2 （如何处理这种异常情况）

else:  （try succ && 上面 except 语句任一满足 ）之外的情况处理方法

  .....

finally:

    .... 无论什么情况都会的处理

"""

def bank(n):
    n.value += 100
    n.value -= 100

def run(n, lock):
    for i in range(1000000):
        lock.acquire()
        try:
            bank(n)
        finally:
            lock.release()

if __name__ == '__main__':
    # t1 = td.Thread(target=run, args=(100,))
    # t2 = td.Thread(target=run, args=(200,))
    lock = mp.Lock()
    v = mp.Value('i', 0)
    t1 = mp.Process(target=run, args=(v, lock))
    t2 = mp.Process(target=run, args=(v, lock))
    t1.start()
    t2.start()
    t1.join()
    t2.join()
    print(v.value)

0
